## Imports and Initialization

## Configurations

## Create Model with Weights

## Get Mask
    Pass in image. Get results of image. Get specific mask from results, and get the non-zero coordinates of the mask. Initialize an fully white image as the mask. Set all of the coordinates to black.

## Inpaint Image
    
    Pass in image and mask. Get coordinates of non-zero pixels of mask. Initialize a new image to be a copy of the image passed in. Also get bincounts of coordinates and iterate through the amount of coordinates. Set a temporary mask to be fully white and and set it black on one strand of the pixels. Inpaint using the mask and the new image.

## Run Object Detection

    Get files, and mask. Inpaint image.